In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Gradient Boosting

In [2]:
#Preparacion de los datos que se usaran

#Apertura de archivo csv a un dataframe de pandas
#revisar la ruta para tu caso particular
ruta = "c:\\Users\\jrgui\\Documents\\GitHub\\Machine_Learning\\Redes_Neuronales\\datasets\\Churn_Modelling.csv"
dataset = pd.read_csv(ruta)
dataset.head(5)
#Datos de los clintes de un banco, queremos entrenar una red que diga si el cliete se va o se queda

# Preprocesado de datos
from sklearn import preprocessing
#Se escogen las variable dependiente e independientes
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values

#Transformar a variables categoricas ordinales
labelencoder_x_paises = preprocessing.LabelEncoder()
#cambio los nombre de los paises en categorias ordinales
X[:,1] = labelencoder_x_paises.fit_transform(X[:,1])
labelencoder_x_generos = preprocessing.LabelEncoder()
#cambio los generos en categorias ordinales
X[:,2] = labelencoder_x_generos.fit_transform(X[:,2])
#Ahorra se debe pasar a variable dummy con onehotencoder
#tranforma el arreglo en un arreglo de numpy
data = np.array(X)
# Separar la primera columna del resto
paises_column = data[:, 1].reshape(-1, 1)
generos_column = data[:, 2].reshape(-1, 1)
demas_columns = data[:, 3:]
primera_columns = data[:, 0].reshape(-1, 1)

# Crear el codificador OneHotEncoder
encoder_paises = preprocessing.OneHotEncoder(sparse_output=False)
encoder_generos = preprocessing.OneHotEncoder(sparse_output=False)
# Ajustar y transformar la primera columna
paises_column_encoded = encoder_paises.fit_transform(paises_column)
generos_column_encoded = encoder_paises.fit_transform(generos_column)
# Concatenar la columna codificada con las demás columnas
X = np.hstack((paises_column_encoded, primera_columns, generos_column_encoded, demas_columns))
#para evitar la colinealidad se deben eliminar 2 columnas una de paises y una de generos
X = np.delete(X, (0,4), axis=1)
#Division de Datos - entrenamiento y validacion
#herramienta para dividir los datos
from sklearn.model_selection import train_test_split
#divide los datos en 20% para la validacion y se colocar una semilla para hacer la division
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

#No es necesario estandarizar los datos

In [8]:
#Algoritmo de Gradient Boosting

from xgboost import XGBClassifier

#juste de Gradient Boosting
clasificador = XGBClassifier()
clasificador.fit(X_train,y_train)

#Predicciones
y_pred = clasificador.predict(X_test)
y_pred = (y_pred>0.5)

#Matriz de confusion
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1494,  101],
       [ 193,  212]], dtype=int64)

In [9]:
#Prueba de validacion cruzada
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = clasificador, X = X_train, y = y_train, cv = 10)
print("Promedio de Precision Global",accuracies.mean())
print("Desviacion Estandar de las Precisiones",accuracies.std())

Promedio de Precision Global 0.8515
Desviacion Estandar de las Precisiones 0.009516433155337116
